## Notebook to Calculate Game Goal Differntial from Kaggle and EH data
#### 2007/2008-2019/2020 are from Kaggle, 2021- is from EH via Kevin

### 2007-2019
- open the team_game_stats csv and group by game_id
- for each game_id, get the home team, away team, home score, away score

### 2020-2022
- for each season, we have a csv with all the goals. 
- open the file, group it by game_id, and use groupby to get the sum of home/away goals that were not 1v0 or 0v1 gamestate (not shootout)
- add the game_id, home team, away team, home score, away score to a csv

In [1]:
import pandas as pd

In [2]:
pre_19_game_df = pd.read_csv('/Users/tylerviducic/dev/hockey_analytics/gamescore_model/kaggle_data/game.csv')
print(pre_19_game_df.head().to_markdown())

|    |    game_id |   season | type   | date_time_GMT        |   away_team_id |   home_team_id |   away_goals |   home_goals | outcome      | home_rink_side_start   | venue          | venue_link          | venue_time_zone_id   |   venue_time_zone_offset | venue_time_zone_tz   |
|---:|-----------:|---------:|:-------|:---------------------|---------------:|---------------:|-------------:|-------------:|:-------------|:-----------------------|:---------------|:--------------------|:---------------------|-------------------------:|:---------------------|
|  0 | 2016020045 | 20162017 | R      | 2016-10-19T00:30:00Z |              4 |             16 |            4 |            7 | home win REG | right                  | United Center  | /api/v1/venues/null | America/Chicago      |                       -5 | CDT                  |
|  1 | 2017020812 | 20172018 | R      | 2018-02-07T00:00:00Z |             24 |              7 |            4 |            3 | away win OT  | left                 

In [3]:
pre_19_game_team_stats_df = pd.read_csv('/Users/tylerviducic/dev/hockey_analytics/gamescore_model/kaggle_data/game_teams_stats.csv')
print(pre_19_game_team_stats_df.head().to_markdown())

|    |    game_id |   team_id | HoA   | won   | settled_in   | head_coach       |   goals |   shots |   hits |   pim |   powerPlayOpportunities |   powerPlayGoals |   faceOffWinPercentage |   giveaways |   takeaways |   blocked | startRinkSide   |
|---:|-----------:|----------:|:------|:------|:-------------|:-----------------|--------:|--------:|-------:|------:|-------------------------:|-----------------:|-----------------------:|------------:|------------:|----------:|:----------------|
|  0 | 2016020045 |         4 | away  | False | REG          | Dave Hakstol     |       4 |      27 |     30 |     6 |                        4 |                2 |                   50.9 |          12 |           9 |        11 | left            |
|  1 | 2016020045 |        16 | home  | True  | REG          | Joel Quenneville |       7 |      28 |     20 |     8 |                        3 |                2 |                   49.1 |          16 |           8 |         9 | left            |
|  2 | 2

In [4]:
team_stats_grouped = pre_19_game_team_stats_df.groupby('game_id')

In [5]:
away_df = pd.concat([group.loc[group['HoA'] == 'away', ['game_id', 'team_id', 'goals']].reset_index(drop=True) for game, group in team_stats_grouped])
away_df = away_df.rename(columns={'team_id': 'away_team_id', 'goals': 'away_goals'})
home_df = pd.concat([group.loc[group['HoA'] == 'home', ['game_id', 'team_id', 'goals']].reset_index(drop=True) for game, group in team_stats_grouped])
home_df = home_df.rename(columns={'team_id': 'home_team_id', 'goals': 'home_goals'})
goal_diff_df = pd.merge(away_df, home_df, on='game_id')
goal_diff_df['home_goals'] = goal_diff_df['home_goals'].astype("Int64") # Int64 allows for null values
goal_diff_df['away_goals'] = goal_diff_df['away_goals'].astype("Int64")


In [6]:
print(goal_diff_df.head().to_markdown())

|    |    game_id |   away_team_id |   away_goals |   home_team_id |   home_goals |
|---:|-----------:|---------------:|-------------:|---------------:|-------------:|
|  0 | 2000020001 |             21 |            2 |             25 |            2 |
|  1 | 2000020002 |              9 |            4 |              6 |            4 |
|  2 | 2000020003 |             16 |            2 |              7 |            4 |
|  3 | 2000020004 |             23 |            3 |              4 |            6 |
|  4 | 2000020005 |             17 |            4 |             20 |            3 |


In [7]:
eh_df = pd.read_csv('/Users/tylerviducic/dev/hockey_analytics/gamescore_model/kaggle_data/EH_pbp_query_20202021_2023-08-11.csv')

In [8]:
print(eh_df.head(50).to_markdown())

|    |   season |    game_id | game_date   | session   |   event_index |   game_period |   game_seconds | clock_time   | event_type   | event_description                                                                            | event_detail   | event_zone   | event_team   | event_player_1     | event_player_2   | event_player_3     |   event_length |   coords_x |   coords_y |   num_on |   num_off |   players_on |   players_off | home_on_1          | home_on_2          | home_on_3           | home_on_4           | home_on_5           | home_on_6         |   home_on_7 | away_on_1         | away_on_2         | away_on_3         | away_on_4          | away_on_5         | away_on_6       |   away_on_7 | home_goalie         | away_goalie       | home_team   | away_team   |   home_skaters |   away_skaters |   home_score |   away_score | game_score_state   | game_strength_state   | home_zone   |   pbp_distance |   event_distance |   event_angle |   home_zonestart |   face_index |   pen_inde

In [9]:
eh_df = eh_df[eh_df['game_period'] < 5]

In [10]:
grouped_df = eh_df.groupby('game_id')

In [11]:
for game, group in grouped_df:
    print(group.iloc[-1])
    break
        

season                                                          20202021
game_id                                                       2020020001
game_date                                                     2021-01-13
session                                                                R
event_index                                                          520
game_period                                                            3
game_seconds                                                        3154
clock_time                                                          7:26
event_type                                                          GOAL
event_description      PHI #13 HAYES(1), Wrist, Off. Zone, 13 ft.Assi...
event_detail                                                       Wrist
event_zone                                                           Off
event_team                                                           PHI
event_player_1                                     

In [16]:
def get_team_id(team_name):
    eh_team_id_dict = {
    'N.J': 1,
    'PHI': 4,
    'L.A': 26,
    'T.B': 14,
    'BOS': 6,
    'NYR': 3,
    'PIT': 5,
    'DET': 17,
    'S.J': 28,
    'NSH': 18,
    'VAN': 23,
    'CHI': 16,
    'OTT': 9,
    'MTL': 8,
    'MIN': 30,
    'WSH': 15,
    'STL': 19,
    'ANA': 24,
    'PHX': 27,
    'NYI': 2,
    'TOR': 10,
    'FLA': 13,
    'BUF': 7,
    'CGY': 20,
    'COL': 21,
    'DAL': 25,
    'CBJ': 29,
    'WPG': 52,
    'EDM': 22,
    'VGK': 54,
    'CAR': 12,
    'ARI': 53,
    'ATL': 11
    }
    return eh_team_id_dict[team_name]

def get_final_scores(df):
    labels = ['game_id','away_team', 'away_score', 'home_team', 'home_score', ]
    grouped_df = df.groupby('game_id')
    last_row = pd.concat([group.iloc[[-1]] for game, group in grouped_df], ignore_index=True).loc[:, labels]
    last_row = last_row.rename(columns={'home_team': 'home_team_id', 'away_team': 'away_team_id'})
    last_row['home_team_id'] = last_row['home_team_id'].apply(get_team_id)
    last_row['away_team_id'] = last_row['away_team_id'].apply(get_team_id)
    return last_row


In [17]:
final_scores_df = get_final_scores(eh_df)

In [21]:
print(final_scores_df.head(10).to_markdown())

|    |     game_id |   away_team_id |   away_score |   home_team_id |   home_score |
|---:|------------:|---------------:|-------------:|---------------:|-------------:|
|  0 | 2.02002e+09 |              5 |            3 |              4 |            5 |
|  1 | 2.02002e+09 |             16 |            0 |             14 |            5 |
|  2 | 2.02002e+09 |              8 |            4 |             10 |            4 |
|  3 | 2.02002e+09 |             23 |            4 |             22 |            3 |
|  4 | 2.02002e+09 |             19 |            3 |             21 |            1 |
|  5 | 2.02002e+09 |             15 |            5 |              7 |            4 |
|  6 | 2.02002e+09 |              6 |            2 |              1 |            1 |
|  7 | 2.02002e+09 |              2 |            3 |              3 |            0 |
|  8 | 2.02002e+09 |             25 |            1 |             13 |            2 |
|  9 | 2.02002e+09 |             12 |            2 |             

In [20]:
# TODO: over EH data files (or add them together)
# TODO: merge with other file